In [1]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# ===== Paths =====
TRAIN_DIR = r'C:\Users\madha\Downloads\dogvscats\training_set\training_set'
TEST_DIR = r'C:\Users\madha\csv\test1'

# ===== Parameters =====
IMG_SIZE = (64, 64)  # Resize images to 64x64

def load_images_from_folder(folder_path, label):
    data = []
    labels = []
    print(f"Loading '{label}' from {folder_path}")
    for filename in tqdm(os.listdir(folder_path)):
        if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue  # Skip non-image files

        filepath = os.path.join(folder_path, filename)
        try:
            img = cv2.imread(filepath, cv2.IMREAD_COLOR)
            if img is None:
                continue  # Skip if image is unreadable
            img = cv2.resize(img, IMG_SIZE)
            data.append(img.flatten())  # Flatten the image to 1D vector
            labels.append(label)
        except Exception as e:
            print(f"Failed to load {filename}: {e}")
    return data, labels


# ===== Load Training Data =====
cat_data, cat_labels = load_images_from_folder(os.path.join(TRAIN_DIR, 'cats'), 0)
dog_data, dog_labels = load_images_from_folder(os.path.join(TRAIN_DIR, 'dogs'), 1)

X_train = np.array(cat_data + dog_data)
y_train = np.array(cat_labels + dog_labels)

print(f"Loaded {len(X_train)} training images")

# ===== Load Test Data =====
cat_test_data, cat_test_labels = load_images_from_folder(os.path.join(TEST_DIR, 'cats'), 0)
dog_test_data, dog_test_labels = load_images_from_folder(os.path.join(TEST_DIR, 'dogs'), 1)

X_test = np.array(cat_test_data + dog_test_data)
y_test = np.array(cat_test_labels + dog_test_labels)

print(f"Loaded {len(X_test)} test images")

# ===== Standardize Data =====
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ===== Train SVM =====
print("Training SVM...")
svm = SVC(kernel='linear')
svm.fit(X_train_scaled, y_train)

# ===== Predict =====
y_pred = svm.predict(X_test_scaled)

print(y_pred)

# # ===== Evaluation =====
# print("\nClassification Report:")
# print(classification_report(y_test, y_pred, target_names=['Cat', 'Dog']))


Loading '0' from C:\Users\madha\Downloads\dogvscats\training_set\training_set\cats


100%|██████████| 4001/4001 [00:09<00:00, 429.44it/s]


Loading '1' from C:\Users\madha\Downloads\dogvscats\training_set\training_set\dogs


100%|██████████| 4006/4006 [00:05<00:00, 685.02it/s]


Loaded 8005 training images
Loading '0' from C:\Users\madha\csv\test1\cats


100%|██████████| 64/64 [00:00<00:00, 703.19it/s]


Loading '1' from C:\Users\madha\csv\test1\dogs


100%|██████████| 64/64 [00:00<00:00, 609.55it/s]


Loaded 128 test images
Training SVM...
[0 1 1 0 0 0 0 1 1 0 1 1 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1 1 0 0 1 0 0 1 0 0
 0 1 0 0 0 1 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 1 0
 0 1 0 1 0 1 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 0 1 1 0 0 1 0 0
 0 1 0 0 0 0 0 1 0 1 0 1 0 1 1 1 1]


In [2]:
print(classification_report(y_test, y_pred, target_names=['Cat', 'Dog'], labels=[0, 1]))


              precision    recall  f1-score   support

         Cat       0.56      0.59      0.58        64
         Dog       0.57      0.53      0.55        64

    accuracy                           0.56       128
   macro avg       0.56      0.56      0.56       128
weighted avg       0.56      0.56      0.56       128

